# Data Class

In [1]:
import random


class Sentiment:
    NEGATIVE = "NEGATIVE"
    NEUTRAL = "NEUTRAL"
    POSITIVE = "POSITIVE"


class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()

    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else:  #score of 4 or 5
            return Sentiment.POSITIVE


class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews

    def get_text(self):
        return [x.text for x in self.reviews]

    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]

    def evenly_distribution(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))

        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)



# Load Data 

In [2]:
import json

file_name = r'dataBooks_5_new.json'

reviews = []
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        #         print(review['reviewText'])
        #         print(review['overall'])
        reviews.append(Review(review['reviewText'], review['overall']))

reviews[5].text


"Boy, what a whiney person Adrianna is, demanding too. Muscling Adam into going to a party just to make her happy. Each has lusted after the other for 4years. Surprise, surprise she gets them lost in the woods and he has no backbone to stand up to her. I will read the next story, but I'm not sold yet.  Only time will tell."

# Prep Data

In [3]:
from sklearn.model_selection import train_test_split

training, test = train_test_split(reviews, test_size=0.33, random_state=42)

train_container = ReviewContainer(training)
test_container = ReviewContainer(test)


In [4]:
train_container.evenly_distribution()
train_x = train_container.get_text()
train_y = train_container.get_sentiment()

test_container.evenly_distribution()
test_x = test_container.get_text()
test_y = test_container.get_sentiment()

print(train_y.count(Sentiment.POSITIVE))
print(train_y.count(Sentiment.NEGATIVE))

21779
21779


## Bag of words vectorization 

In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = TfidfVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)

test_x_vectors = vectorizer.transform(test_x)

print(train_x[0])
print(train_x_vectors[0].toarray())



The Dream Dress was such an interesting idea for a story. Usually in a romantic book, the story is a build up to the proposal/wedding. Here, we get to experience the fun and frustrations of planning weddings right off the bat, just not that of the main character. We get to experience the stress and strain of wedding through that of Gabi, a seamstress at a high end wedding dress boutique, who dreams of one day being able to create and sale her own designs.I also enjoyed this story for another reason. Yes, there is a budding romance between Jordan and Gabi, but the love story in this book focused more on Gabi discovering God's love for her. Throughout the book, Gabi is faced with hard choices in her job and personal life and through these choices she grows as a person and slowly discovers what God has planned for her and her life. That part of the story really touched me.It was a little hard to get settled into the story at the beginning. Gabi has low self esteem. It's hard for me to con

# Classification

### Linear SVM

In [6]:
from sklearn import svm

clf_svm = svm.SVC(kernel='linear')

clf_svm.fit(train_x_vectors, train_y)

test_x[0]

clf_svm.predict(test_x_vectors[0])


array(['NEGATIVE'], dtype='<U8')

### Decision Tree

In [7]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_y)

clf_dec.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

### Naive Bayes

In [8]:
from sklearn.naive_bayes import GaussianNB

clf_gnb = DecisionTreeClassifier()
clf_gnb.fit(train_x_vectors, train_y)

clf_gnb.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

### Logistic Regression

In [9]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()

clf_log.fit(train_x_vectors, train_y)

clf_log.predict(test_x_vectors[0])

C:\Users\rohit\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array(['POSITIVE'], dtype='<U8')

# Evaluation

In [10]:
# Mean Accuracy
print(clf_svm.score(test_x_vectors, test_y))
print(clf_dec.score(test_x_vectors, test_y))
print(clf_gnb.score(test_x_vectors, test_y))
print(clf_log.score(test_x_vectors, test_y))

0.904173685195583
0.7363840539023021
0.7345124461912783
0.902676399026764


In [11]:
# F1 scores
from sklearn.metrics import f1_score

# f1_score(test_y, clf_svm.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE])
f1_score(test_y, clf_svm.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE])



array([0.90407494, 0.90427223])

In [12]:
test_set = ['not great', 'i dont like it', 'relax', 'trash', 'absolute junk']
new_test = vectorizer.transform(test_set)

clf_svm.predict(new_test)

array(['POSITIVE', 'NEGATIVE', 'POSITIVE', 'NEGATIVE', 'NEGATIVE'],
      dtype='<U8')

## Tuning our model (with grid search)

In [13]:
# from sklearn.model_selection import GridSearchCV
#
# parameters = {'kernel': ('linear', 'rbf'), 'C': (1, 4, 8, 16, 32)}
#
# svc = svm.SVC()
# clf = GridSearchCV(svc, parameters, cv=5)
# clf.fit(train_x_vectors, train_y)




In [14]:
# print(clf.score(test_x_vectors, test_y))

## Saving Model

In [18]:
import pickle

with open(r'./models/sentiment_classifier.pkl', 'wb') as f:
    pickle.dump(clf_svm, f)

### load model

In [19]:
import pickle

with open(r'./models/sentiment_classifier.pkl', 'rb') as f:
    loaded_clf = pickle.load(f)

In [20]:
# print(test_x[56])

t1 = ['there was nothing interesting in the book, waste of time', 'absolute disgusting']
test_case = vectorizer.transform(t1)
loaded_clf.predict(test_case)

array(['NEGATIVE', 'NEGATIVE'], dtype='<U8')